### Libraries

In [8]:
import os
import pickle
import numpy
from tqdm.notebook import tqdm

#ML Modules
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical,plot_model
from tensorflow.keras.layers import Input,Dense,LSTM,Embedding,Dropout,add
from tensorflow.keras.applications.vgg16 import preprocess_input

### Directory Structure

In [6]:
BASE_DIR="./Flickr8kDataset/"
WORKING_DIR="/"

### Extract Image Features

In [5]:
#Loading the VGG16 Model
model=VGG16()
# Restructure the model
model=Model(inputs=model.inputs,outputs=model.layers[-2].output)
# Summarize the model
print(model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [10]:
# Extracting features from the image
features={}
directory=os.path.join(BASE_DIR,"Images")
for img_name in tqdm(os.listdir(directory)):
  # Load the image from file
  imagepath=directory+'/'+img_name
  image=load_img(imagepath,target_size=(224,224))
  # Convert Image pexels to numpy array
  image=img_to_array(image)
  # Reshape Data for model
  image=image.reshape(1,image.shape[0],image.shape[1],image.shape[2])
  #Preprocess for VGG
  image=preprocess_input(image)
  #extract Features
  feature=model.predict(image,verbose=0)
  #Image ID
  image_id=img_name.split(".")[0]
  #Store the feature in Features dict
  features[image_id]=feature

  0%|          | 0/8091 [00:00<?, ?it/s]

2022-11-02 18:42:25.723380: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-02 18:42:25.793333: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


### Saving Features Permanently

In [14]:
#Storing data in Pickle File
pickle.dump(features,open("Flickr8kFeatures.pickle",'wb'))

In [15]:
#Load features from pickle file
with open("Flickr8kFeatures.pickle","rb") as f:
  features=pickle.load(f)


### Loading the Captions Data

In [18]:
with open(os.path.join(BASE_DIR, 'captions.txt'), 'r') as f:
    next(f)
    captions_doc = f.read()

In [19]:
# create mapping of image to captions
mapping = {}
# process lines
for line in tqdm(captions_doc.split('\n')):
    # split the line by comma(,)
    tokens = line.split(',')
    if len(line) < 2:
        continue
    image_id, caption = tokens[0], tokens[1:]
    # remove extension from image ID
    image_id = image_id.split('.')[0]
    # convert caption list to string
    caption = " ".join(caption)
    # create list if needed
    if image_id not in mapping:
        mapping[image_id] = []
    # store the caption
    mapping[image_id].append(caption)

  0%|          | 0/40456 [00:00<?, ?it/s]

In [20]:
len(mapping)

8091

In [21]:
def clean(mapping):
  for key,captions in mapping.items():
    for i in range(len(captions)):
      #One single Caption
      caption=caption[i]

SyntaxError: unexpected EOF while parsing (2388550525.py, line 1)